<a href="https://colab.research.google.com/github/carvalheirafc/deep_learning_stuff/blob/master/mlp/diabetes/task3/feed_foward_with_placeholders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [0]:
csv_file = 'https://raw.githubusercontent.com/carvalheirafc/deep_learning_stuff/master/mlp/diabetes/diabetes_pca_result.csv'
df = pd.read_csv(csv_file)
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Component 1,768.0,-6.693141e-17,1.711960,-5.587340,-1.027072,0.152931,1.095689,5.963579
Component 2,768.0,-6.071532e-17,1.318184,-2.955806,-1.007096,-0.225966,0.959128,3.606582
Component 3,768.0,-3.324887e-17,1.015629,-3.203088,-0.646496,-0.103940,0.549774,4.754503
Component 4,768.0,2.349105e-18,0.966941,-2.192185,-0.664677,-0.115829,0.543242,4.125987
Diagnóstico,768.0,3.489583e-01,0.476951,0.000000,0.000000,0.000000,1.000000,1.000000


In [0]:
features = df.drop('Diagnóstico', axis=1).values
labels = df['Diagnóstico'].values

print(features.shape)
print(labels.shape)

(768, 4)
(768,)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    labels, 
                                                    test_size=0.33, 
                                                    random_state=666)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(514, 4)
(254, 4)
(514,)
(254,)


In [0]:
labels_train = (np.arange(2) == y_train[:,None]).astype(np.float32)
labels_test = (np.arange(2) == y_test[:,None]).astype(np.float32)

In [0]:
inputs = tf.placeholder(tf.float32, shape=(None, X_train.shape[1]), name='inputs')
label = tf.placeholder(tf.float32, shape=(None, 2), name='labels')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# First layer
hidden_size_one = 128

weight_one = tf.Variable(tf.random_normal([hidden_size_one, 
                                           X_train.shape[1]], stddev=0.01), 
                         name='weight_one')

bias_one = tf.Variable(tf.constant(0.1, shape=(hidden_size_one, 1)), 
                       name='bias_one')

y_one = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(weight_one, 
                                                  tf.transpose(inputs)), 
                                        bias_one)), keep_prob=0.5)

In [0]:
# Second layer

hidden_size_two = 256
weight_two = tf.Variable(tf.random_normal([hidden_size_two, 
                                           hidden_size_one], 
                                          stddev=0.01), 
                         name='weight_two')

bias_two = tf.Variable(tf.constant(0.1, shape=(hidden_size_two, 1)),
                       name='bias_two')

y_two = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(weight_two, 
                                                  y_one), 
                                        bias_two)), keep_prob=0.5)

In [0]:
# Output layer
weight_out = tf.Variable(tf.random_normal([2, 
                                           hidden_size_two], stddev=0.01), 
                         name='weight_out')

bias_out = tf.Variable(tf.random_normal([2, 1]), name='bias_out')
y_out = tf.transpose(tf.add(tf.matmul(weight_out, y_two), bias_out))

In [0]:
# Loss function and optimizer
lr = tf.placeholder(tf.float32, shape=(), name='learning_rate')
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_out, labels=label))
optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

# Prediction
pred = tf.nn.softmax(y_out)
pred_label = tf.argmax(pred, 1)
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
# Create operation which will initialize all variables
init = tf.global_variables_initializer()

# Configure GPU not to use all memory
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Start a new tensorflow session and initialize variables
sess = tf.InteractiveSession(config=config)
sess.run(init)


# This is the main training loop: we train for 50 epochs with a learning rate of 0.05 and another 
# 50 epochs with a smaller learning rate of 0.01
for learning_rate in [0.05, 0.01]:
    for epoch in range(50):
        avg_cost = 0.0

        # For each epoch, we go through all the samples we have.
        for i in range(X_train.shape[0]):
            # Finally, this is where the magic happens: run our optimizer, feed the current example into X and the current target into Y
            _, c = sess.run([optimizer, loss], feed_dict={lr:learning_rate, 
                                                          inputs: X_train[i, None],
                                                          label: labels_train[i, None]})
            avg_cost += c
        avg_cost /= X_train.shape[0]    

        # Print the cost in this epcho to the console.
        if epoch % 10 == 0:
            print("Epoch: {:3d}    Train Cost: {:.4f}".format(epoch, avg_cost))

Epoch:   0    Train Cost: 0.6253
Epoch:  10    Train Cost: 0.5919
Epoch:  20    Train Cost: 0.5669
Epoch:  30    Train Cost: 0.5720
Epoch:  40    Train Cost: 0.5867
Epoch:   0    Train Cost: 0.5435
Epoch:  10    Train Cost: 0.5172
Epoch:  20    Train Cost: 0.5100
Epoch:  30    Train Cost: 0.5117
Epoch:  40    Train Cost: 0.4987


In [0]:
acc_train = accuracy.eval(feed_dict={inputs: X_train, label: labels_train})
print("Train accuracy: {:3.2f}%".format(acc_train*100.0))

acc_test = accuracy.eval(feed_dict={inputs: X_test, label: labels_test})
print("Test accuracy:  {:3.2f}%".format(acc_test*100.0))

Train accuracy: 76.46%
Test accuracy:  72.05%
